# RRT Pipeline - Google Colab Setup

Dit notebook configureert de omgeving voor het draaien van de RL pipeline op Google Colab.

**Stappen:**
1. GPU checken
2. Dependencies installeren
3. Code clonen van GitHub
4. Google Drive mounten (voor data)
5. Config aanpassen voor Colab paths
6. Pipeline runnen

## 1. GPU Check

In [ ]:
# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("WARNING: No GPU found! Go to Runtime > Change runtime type > GPU")

## 2. Dependencies installeren

In [ ]:
!pip install d3rlpy optuna pandas matplotlib seaborn pyyaml scikit-learn tqdm -q
print("Dependencies installed!")

## 3. Code clonen van GitHub

In [ ]:
# Clone repository
GITHUB_REPO = "https://github.com/karel2324/Thesis.git"

import os
if not os.path.exists('Thesis'):
    !git clone {GITHUB_REPO}
    print("Repository cloned!")
else:
    print("Repository already exists, pulling latest...")
    !cd Thesis && git pull

In [ ]:
# Navigeer naar code directory
%cd Thesis/RRT_start/2_Total
!ls -la

## 4. Google Drive mounten (voor data)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted!")

In [ ]:
# Check data location (pas aan naar jouw folder structuur)
# Verwachte structuur in Google Drive:
#   MyDrive/Thesis/Data/Shared/
#     aumc/
#       7_days_8_hours_kdigo2/
#         MDP/
#         ICUFD/
#     mimic/
#       7_days_8_hours_kdigo2/
#         MDP/
#         ICUFD/

DATA_BASE_PATH = "/content/drive/MyDrive/Thesis/Data/Shared"  # <-- AANPASSEN!

import os
if os.path.exists(DATA_BASE_PATH):
    print(f"Data folder found: {DATA_BASE_PATH}")
    !ls -la {DATA_BASE_PATH}
else:
    print(f"ERROR: Data folder not found at {DATA_BASE_PATH}")
    print("Upload je data naar Google Drive of pas DATA_BASE_PATH aan.")

## 5. Config aanpassen voor Colab

In [ ]:
import yaml

# Load config
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Update paths for Colab
config['paths']['base_dir'] = DATA_BASE_PATH

# Save updated config
with open('config.yaml', 'w') as f:
    yaml.dump(config, f, default_flow_style=False, sort_keys=False)

print("Config updated with Colab paths!")
print(f"Base directory: {config['paths']['base_dir']}")

## 6. Test imports

In [ ]:
# Test that all imports work
try:
    from utils import load_config, get_data_paths
    from rl_utils import compute_mc_return, evaluate_algo
    print("All imports successful!")
    
    # Test config loading
    config = load_config()
    paths = get_data_paths(config)
    print(f"\nConfigured databases: {list(paths.keys())}")
    
except Exception as e:
    print(f"Import error: {e}")

## 7. Run Pipeline

Pas eerst `config.yaml` aan voor welke stappen je wilt runnen:
- `hpo: true` voor HPO
- `evaluation: true` voor evaluatie
- etc.

In [ ]:
# Run the full pipeline
!python run_pipeline.py

## Alternatief: Run individuele stappen

In [ ]:
# HPO only
# !python _7_hpo.py

In [ ]:
# Evaluation only
# !python _9_evaluation.py

In [ ]:
# External validation only
# !python _10_external_validation.py

## 8. Resultaten downloaden

In [ ]:
# Resultaten worden automatisch opgeslagen in Google Drive
# Check de output folders:
!ls -la {DATA_BASE_PATH}/aumc/7_days_8_hours_kdigo2/ICUFD/

In [ ]:
# Of download specifieke files lokaal
from google.colab import files
# files.download('path/to/results.csv')